# Building Fast Queries On Laptop Data

Data taken from https://www.kaggle.com/ionaskel/laptop-prices.

## Build Class For Lookup

In [2]:
import csv

class Inventory():
    
    def __init__(self, csv_filename):
        
        with open(csv_filename) as f:
            reader = csv.reader(f)
            rows = list(reader)
            self.header = rows[0]
            self.rows = rows[1:]
            
            # convert price column to integer
            for row in self.rows:
                row[-1] = int(row[-1])
                
            # {key:value} == {laptop id: row}
            self.id_to_row = {}
            
            for row in self.rows:
                self.id_to_row[row[0]] = row
            
            # set containing prices of all laptops
            self.prices = set()
            
            for row in self.rows:
                self.prices.add(row[-1])
                
            # sort rows by price    
            def row_price(row):
                return row[-1]
                
            self.rows_by_price = sorted(self.rows, key=row_price)
    
    # return row of laptop (argument = laptop id) 
    # O(N) Time Complexity
    def get_laptop_from_id(self, laptop_id):
        
        for row in self.rows:
            
            if laptop_id == row[0]:
                return row
        
        return None
    
    # return row of laptop when given identifier or id 
    # O(1) Time Complexity
    def get_laptop_from_id_fast(self, laptop_id):
        
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        
        return None
    
    # return True if 1-2 laptops can be bought using exact dollars given
    # O(N^2) Time Complexity
    def check_promotion_dollars(self, dollars):
        
        for x in range(len(self.rows)):
            
            if self.rows[x][-1] == dollars:
                return True
            
            for y in range(x, len(self.rows)):
                
                if self.rows[x][-1] + self.rows[y][-1] == dollars:
                    return True
                
        return False
    
    # return True if 1-2 laptops can be bought using exact dollars given
    # O(N) Time Complexity
    def check_promotion_dollars_fast(self, dollars):
        
        if dollars in self.prices:
            return True
        
        for price in self.prices:
            
            if (dollars - price) in self.prices:
                return True
            
        return False
    
    # return sorted row index of the first more expensive laptop given price
    # Nlog(N) Time Complexity
    def find_first_laptop_more_expensive(self, price):
        left_index = 0
        right_index = len(self.rows_by_price) - 1
        
        while left_index <= right_index:
            mid_index = (left_index + right_index) // 2
            mid_price = self.rows_by_price[mid_index][-1]
            
            if mid_price == price:
                
                for x in range(mid_index, len(self.rows_by_price)):
                    
                    if self.rows_by_price[x][-1] > price:
                        return x
                
                break
            elif mid_price > price:
                right_index = mid_index - 1 
            else:
                left_index = mid_index + 1
                
        return -1

## Initialize Class

In [3]:
inv = Inventory("laptops.csv")

## Speed Test

* get_laptop_from_id
* get_laptop_from_id_fast

In [4]:
import time
import random

ids = [str(random.randint(1000000, 9999999)) for _ in range(10000)]

total_time_no_dict = 0

for val in ids:
    start = time.time()
    inv.get_laptop_from_id(val)
    end = time.time()
    total_time_no_dict += end - start
    
total_time_dict = 0

for val in ids:
    start = time.time()
    inv.get_laptop_from_id_fast(val)
    end = time.time()
    total_time_dict += end - start
    
print(total_time_no_dict)
print(total_time_dict)

1.3705182075500488
0.0060694217681884766


## Speed Test

* check_promotion_dollars
* check_promotion_dollars_fast

In [5]:
prices = [random.randint(100, 5000) for _ in range(100)]

total_time_no_set = 0

for val in prices:
    start = time.time()
    inv.check_promotion_dollars(val)
    end = time.time()
    total_time_no_set += end - start
    
total_time_set = 0

for val in prices:
    start = time.time()
    inv.check_promotion_dollars_fast(val)
    end = time.time()
    total_time_set += end - start
    
print(total_time_no_set)
print(total_time_set)

2.4370601177215576
0.0009052753448486328
